In [13]:
from selenium import webdriver
from tqdm import tqdm
from selenium.webdriver.chrome.options import Options
import json

chromeOptions = Options()
chromeOptions.headless = True
chromeOptions.add_argument("--headless")

class Grow:
    def __init__(self) -> None:
        self.get_driver()
        self.baseIndex = "https://groww.in/mutual-funds/filter?q=&fundSize=&category=%5B%22Equity%22%5D&pageNo={}&sortBy=3"

    def get_driver(self):
        self.driver = webdriver.Chrome(executable_path="./chromedriver", options=chromeOptions)
    def visit(self, url = ""):
        self.driver.get(url)
    def close_driver(self):
        self.driver.close()
    def funds_url(self, n = 1):
        self.fundsUrl = []
        for i in tqdm(range(n)):
            try:
                self.visit(self.baseIndex.format(i))
                table = self.driver.find_element_by_class_name("tb10Table")
                aTags = table.find_elements_by_tag_name("a")
                for a in aTags:
                    link = a.get_attribute('href')
                    self.fundsUrl.append(link)
            except:
                break
    def get_fund_distribution(self):
        self.funds = {}
        index = 0
        for fund in self.fundsUrl:
            self.driver.get(fund)
            div = self.driver.find_element_by_class_name("holdings101TableContainer")
            divN = div.find_element_by_xpath("//div[contains(@class, 'holdings101Cta cur-po') and text()='See All']")
            divN.click()
            rows = div.find_elements_by_class_name('holdings101Row')
            table = []
            for row in rows:
                cols = []
                td = row.find_elements_by_tag_name('td')
                for col in td:
                    cols.append(col.text)
                table.append(cols)
            
            self.funds[fund.split("/")[-1]] = table
            index += 1
            if((index % 10 == 0) and (index != 0)):
                self.close_driver()
                self.get_driver()




        

In [2]:
grow = Grow()

In [3]:
grow.funds_url()

100%|██████████| 1/1 [00:05<00:00,  5.72s/it]


In [4]:
grow.get_fund_distribution()

In [5]:
grow.funds

{'icici-prudential-technology-fund-direct-growth': [['Infosys Ltd.',
   'Technology',
   'Equity',
   '34.6%'],
  ['Tata Consultancy Services Ltd.', 'Technology', 'Equity', '18.0%'],
  ['HCL Technologies Ltd.', 'Technology', 'Equity', '8.6%'],
  ['Bharti Airtel Ltd.', 'Communication', 'Equity', '8.2%'],
  ['Wipro Ltd.', 'Technology', 'Equity', '4.3%'],
  ['Tech Mahindra Ltd.', 'Technology', 'Equity', '3.1%'],
  ['LTIMindtree Ltd.', 'Technology', 'Equity', '1.9%'],
  ['Zee Entertainment Enterprises Ltd.', 'Services', 'Equity', '1.5%'],
  ['Accenture PLC', 'Others', 'Forgn. Eq', '1.1%'],
  ['Indiamart Intermesh Ltd.', 'Services', 'Equity', '0.9%'],
  ['Mphasis Ltd.', 'Technology', 'Equity', '0.9%'],
  ['Bharti Airtel Ltd. - (Partly Paid up Equity Shares (Rights Issue))',
   'Communication',
   'PPE',
   '0.8%'],
  ['Oracle Financial Services Software Ltd.', 'Technology', 'Equity', '0.7%'],
  ['Siemens Ltd.', 'Capital Goods', 'Equity', '0.7%'],
  ['eClerx Services Ltd.', 'Services', 'Equi

In [14]:
with open("file.txt", 'w') as f:
    json.dump(grow.funds, f)

In [10]:
os.getcwd()

'/Users/gurvinderyadav/Documents/mutual-fund-recommender/src'